# Support Email Classification Project part 2


## In this notebook modelling part is covered and the end model is saved as pickle file 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import re

In [ ]:
#Importing cleaned dataset
support_email_data_9_col_2= pd.read_csv(r'/content/drive/MyDrive/fully_cleaned_email_data.csv',encoding= 'latin-1')
support_email_data_9_col_2= support_email_data_9_col_2[['Request_Type','cleaned_body']]

## Preprocessing

### 1. Removed rows with nan values 
### 2. Removed the non-required categories(*Booking Query*, *Promotions*, *Eticket*/*Voucher*, *Booking Information*) 

In [ ]:
#Removing rows with nan  and not required categories 
support_email_data_9_col= support_email_data_9_col[support_email_data_9_col['Request_Type'].notna()]
support_email_data_9_col= support_email_data_9_col[support_email_data_9_col['cleaned_body'].notna()]
support_email_data_9_col= support_email_data_9_col[support_email_data_9_col.Request_Type != 'Booking Query']
support_email_data_9_col= support_email_data_9_col[support_email_data_9_col.Request_Type != 'Promotions']
support_email_data_9_col= support_email_data_9_col[support_email_data_9_col.Request_Type != 'Eticket/Voucher']
support_email_data_9_col= support_email_data_9_col[support_email_data_9_col.Request_Type != 'Booking information']

##Final Categories Taken Are 
### 1. Refund 
### 2. Others
### 3. Amendments
### 4. Cancellation
### 5. Website Error

In [ ]:
ax = support_email_data_9_col['Request_Type'].value_counts().plot(kind='bar', figsize=(19,9), title="Fuck your feelings")
ax.set_xlabel("Request_Type")
ax.set_ylabel("Frequency")

# Modelling

## Steps Inovolved
### 1. Formed a pipline which will do the following processes 
### 2. Formation of Tfidf vectors 
### 3. Used Chi2 for best feature selection
### 4. Random forest for classification

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
import nltk
import copy
%matplotlib inline







df=copy.deepcopy(support_email_data_9_col)
vectorizer = TfidfVectorizer(min_df= 1,max_df= .94, stop_words="english", sublinear_tf=True, norm='l1', ngram_range=(1, 1))
final_features = vectorizer.fit_transform(df['cleaned_body']).toarray()
final_features.shape  

    
# this block is to split the dataset into training and testing set 
X = df['cleaned_body']
df['label_num']  =  df['Request_Type'].map({'Refunds': 0, 'Cancellation': 1,'Others': 2,'Amendment': 3, 'Website Error':4})
#df['label_num'] = df['Request_Type'].factorize()[0] #class embedding
Y = df['label_num']
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0, test_size=0.25)

# instead of doing these steps one at a time, we can use a pipeline to complete then all at once
pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1000)),
                     ('clf', RandomForestClassifier())])

# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train, y_train)
with open('RandomForest.pickle', 'wb') as f:
    pickle.dump(model, f)

ytest = np.array(y_test)

# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))

### Saving model as a pickle file 

In [ ]:
# save
import pickle
with open('classificationModel.pkl','wb') as f:
    pickle.dump(model,f)


In [ ]:
# load
import pickle
with open('classificationModel.pkl', 'rb') as f:
  clf2 = pickle.load(f)